In [1]:
%matplotlib inline
import numpy as np
import scipy.ndimage
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import matplotlib
import xarray as xr
import pandas as pd
import dask
import os.path
from __future__ import division
matplotlib.rcParams['figure.figsize'] = (8.0, 6.0)


from useful import *

global data_dir
data_dir = '/scratch/ap587/dry_data/'

In [9]:
def testfun(exp):
    runs = range(0,5)
    runset = open_runset(data_dir, exp,runs)    
    
    runset.drop(['ps','vor','div']) 
    c = runset.mean('lon')
    c.time.data = c.time.data/24
    print(c)
    #c.load()
    
    #data = {'omega':c.omega.data, 'vcomp':c.vcomp.data, 'time':c.time.data, 'pfull':c.pfull.data, 'lat':c.lat.data}
    #psi = cal_stream_fn(data)
    #c['psi'] = (('time','pfull','lat'),psi)
    
    do_log = 0
    vmaxt = 300
    vmint = 200
    vmaxu = 25
    vmaxsf = 0.5e11
    
    
    
    #plot_timeseries(c.teq, 'exp'+exp+'teq timeseries')
    #plot_timeseries(c.temp, 'exp'+exp+'temp timeseries')
    #plot_grid_sf(c, 'Annual Mean for exp' + exp, do_log=do_log, vmaxt=vmaxt, vmint=vmint,vmaxu=vmaxu,vmaxsf=vmaxsf)
     
    return c


    
exp = '0_test1/'
c = testfun(exp)


<xarray.Dataset>
Dimensions:  (lat: 64, latb: 65, lonb: 129, pfull: 30, phalf: 31)
Coordinates:
  * phalf    (phalf) float64 0.0 33.33 66.67 100.0 133.3 166.7 200.0 233.3 ...
  * latb     (latb) float64 -90.0 -86.58 -83.76 -80.96 -78.16 -75.36 -72.57 ...
  * lonb     (lonb) float64 -1.406 1.406 4.219 7.031 9.844 12.66 15.47 18.28 ...
  * pfull    (pfull) float64 12.26 49.05 82.77 116.3 149.7 183.1 216.5 249.8 ...
  * lat      (lat) float64 -87.86 -85.1 -82.31 -79.53 -76.74 -73.95 -71.16 ...
    time     float64 351.0
Data variables:
    ps       (lat) float64 9.154e+04 9.184e+04 9.238e+04 9.311e+04 9.395e+04 ...
    temp     (pfull, lat) float64 173.3 173.5 174.0 174.7 175.8 177.2 178.9 ...
    ucomp    (pfull, lat) float64 5.315 12.49 20.3 28.72 37.4 45.66 52.9 ...
    vcomp    (pfull, lat) float64 0.001791 0.00163 0.004415 0.01882 0.04361 ...
    bk       (phalf) float64 0.0 0.03333 0.06667 0.1 0.1333 0.1667 0.2 ...
    vor      (pfull, lat) float64 -4.5e-05 -4.701e-05 -4.99e-05 -5.2

In [ ]:
lhalf = np.array([-5.65297628e-05 ,  1.67016276e-04  , 2.82860288e-04  , 3.03908944e-04,
   1.21139983e-04 , -1.66220734e-04,  -4.72422799e-05 ,  3.06479439e-04,
   6.70444187e-04 ,  1.13132098e-03 ,  1.79086721e-03 ,  2.31039372e-03,
   2.59153556e-03 ,  2.71285003e-03 ,  2.74575303e-03 ,  2.70820606e-03,
   2.59352545e-03 ,  2.44352854e-03 ,  2.23414351e-03 ,  2.20708513e-03,
   2.01329314e-03 ,  1.74875298e-03 ,  9.61044465e-04 , -2.07529328e-05,
  -7.97192195e-04 , -1.87087786e-03 , -2.54887406e-03 , -3.92095840e-03,
  -5.32527524e-03  ,-7.78498028e-03 , -1.36227460e-02])
heights = np.array([28405.65598861,  21532.37501293,  18266.71776349,  16009.2898625,
  14274.90899453 , 12864.32857213 , 11674.78075756,  10645.99547379,
   9739.50415659 ,  8929.21906136 ,  8196.61968246 ,  7528.07697183,
   6913.26663812 ,  6344.17870018 ,  5814.47280124 ,  5319.04369608,
   4853.71962368 ,  4415.04755625,   4000.13690421 ,  3606.54355213,
   3232.18234012 ,  2875.26000321 ,  2534.22308265  , 2207.71696702,
   1894.55332483 ,  1593.68394558 ,  1304.17953286 ,  1025.21236482,
    756.04200533 ,   496.00344328 ,   244.49718102])
phalfk = np.array([ 2.49738788,  3.22632856 , 3.64378766,  3.96353304,  4.22812699 , 4.45629428,
  4.65825201 , 4.84028192 , 5.00655959 , 5.16001928  ,5.30281112 , 5.43656312,
  5.56254094,  5.68175015 , 5.79500437,  5.90297225 , 6.00621064  ,6.10518877,
  6.20030609,  6.29190577 , 6.38028498,  6.46570299 , 6.54838747, 6.62853953,
  6.70633782 , 6.78194179 , 6.85549444,  6.92712454 , 6.99694851  ,7.06507197,
  7.13159109])
lhalf = lhalf[::-1]
heights = heights[::-1]
phalfk=phalfk[::-1]
h_trop = 0
crit_lapse = 2/1000     # critical lapse rate (K/m)
r_depth = 2000          # required depth (m) if lapse rate is below crit_lapse for this depth, tropopause is defined
k = 2/7
g = 9.81
rdgas = 287.05
j = 1
no_trop = 1
has_been_pos = 0
while (no_trop) and (j<len(phalfk)):
    #print('searching for tropopause: %d' % j) 
    #print('lapse rate is: %.4f' % lhalf[j])
    if (lhalf[j] <= crit_lapse) & has_been_pos:        
        is_within_2k = (heights-heights[j]) < r_depth
        is_above =  heights-heights[j] > 0
        ind = is_within_2k & is_above
        is_below_cl = lhalf[ind] < crit_lapse
        if is_below_cl.all():            
            no_trop = 0
            p_tropk = phalfk[j-1] + (phalfk[j] - phalfk[j-1])/(lhalf[j] - lhalf[j-1])*(crit_lapse-lhalf[j-1])
            h_trop = rdgas*260/g*np.log(1000/p_tropk**(1/k))
            #print('Found tropopause, height: %.2f' % h_trop)
    if lhalf[j] > 0:
        has_been_pos = 1
    j += 1


print heights
print lhalf

In [30]:
d = c.isel(time=300)

print c

<xarray.Dataset>
Dimensions:  (lat: 64, latb: 65, lonb: 129, pfull: 30, phalf: 31, time: 360)
Coordinates:
  * phalf    (phalf) float64 0.0 33.33 66.67 100.0 133.3 166.7 200.0 233.3 ...
  * latb     (latb) float64 -90.0 -86.58 -83.76 -80.96 -78.16 -75.36 -72.57 ...
  * lonb     (lonb) float64 -1.406 1.406 4.219 7.031 9.844 12.66 15.47 18.28 ...
  * pfull    (pfull) float64 12.26 49.05 82.77 116.3 149.7 183.1 216.5 249.8 ...
  * lat      (lat) float64 -87.86 -85.1 -82.31 -79.53 -76.74 -73.95 -71.16 ...
  * time     (time) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 ...
Data variables:
    ps       (time, lat) float64 1.003e+05 1.003e+05 1.003e+05 1.002e+05 ...
    temp     (time, pfull, lat) float64 261.0 261.0 261.0 261.0 261.0 261.1 ...
    ucomp    (time, pfull, lat) float64 0.05273 0.1784 0.3872 0.6441 0.9126 ...
    vcomp    (time, pfull, lat) float64 -0.004902 -0.01834 -0.04028 -0.06585 ...
    bk       (time, phalf) float64 0.0 0.03333 0.06667 0.1 0.1333 0.1667 0.2 ...
